In [8]:
# FROM HERE
# https://robotwealth.com/getting-started-with-the-interactive-brokers-native-api/

# %load ibkr-api/account_summary.py
from threading import Thread, Event
import time
from typing import Any
from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.common import *
from ibapi.account_summary_tags import AccountSummaryTags


class ibapp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.done = Event()  # use threading.Event to signal between threads
        self.connection_ready = Event()  # to signal the connection has been established

    # override Ewrapper.error
    def error(
        self, reqId: TickerId, errorCode: int, errorString: str, contract: Any = None
    ):
        print("Error: ", reqId, " ", errorCode, " ", errorString)
        if errorCode == 502:  # not connected
            # set self.done (a threading.Event) to True
            self.done.set()

    # override Ewrapper.accountSummary - method for receiving account summary
    def accountSummary(
        self, reqId: int, account: str, tag: str, value: str, currency: str
    ):
        # just print the account information to screen
        print(
            "AccountSummary. ReqId:",
            reqId,
            "Account:",
            account,
            "Tag: ",
            tag,
            "Value:",
            value,
            "Currency:",
            currency,
        )

    # override Ewrapper.accountSummaryEnd - notifies when account summary information has been received
    def accountSummaryEnd(self, reqId: int):
        # print to screen
        print("AccountSummaryEnd. ReqId:", reqId)
        # set self.done (a threading.Event) to True
        self.done.set()

    # override Ewrapper.nextValidID - used to signal that the connection between application and TWS is complete
    # returns the next valid orderID (for any future transactions)
    # if we send messages before the connection has been established, they can be lost
    # so wait for this method to be called
    def nextValidId(self, orderId: int):
        print(f"Connection ready, next valid order ID: {orderId}")
        self.connection_ready.set()  # signal that the connection is ready


# define our event loop - this will run in its own thread
def run_loop(app):
    app.run()


# instantiate an ibapp
app = ibapp()

# connect
app.connect("127.0.0.1", 7496, clientId=0)  # clientID identifies our application

# start the application's event loop in a thread
api_thread = Thread(target=run_loop, args=(app,), daemon=True)
api_thread.start()

# wait until the Ewrapper.nextValidId callback is triggered, indicating a successful connection
app.connection_ready.wait()

# request account summary
print("Requesting account summary")
type (app.reqAccountSummary(0, "All", AccountSummaryTags.AllTags))
# app.reqAccountSummary(0, "All", AccountSummaryTags.Cushion)
# app.reqAccountSummary(0, "All", AccountSummaryTags.Cushion)


# wait for the account summary to finish (ie block until app.done - a threading.Event - becomes true)
app.done.wait()

# disconnect
app.disconnect()

Connection ready, next valid order ID: 1
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:eufarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is OK:eufarmnj
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK:ushmds
Error:  -1   2158   Sec-def data farm connection is OK:secdefnj
Requesting account summary
AccountSummary. ReqId: 0 Account: DU5417029 Tag:  AccountType Value: INDIVIDUAL Currency: 
AccountSummary. ReqId: 0 Account: DU5417029 Tag:  Cushion Value: 0.141045 Currency: 
AccountSummary. ReqId: 0 Account: DU5417029 Tag:  LookAheadNextChange Value: 1742500800 Currency: 
AccountSummary. ReqId: 0 Account: DU5417029 Tag:  AccruedCash Value: 0.00 Currency: EUR
AccountSummary. ReqId: 0 Account: DU5417029 Tag:  AvailableFunds Value: 79